<a href="https://colab.research.google.com/github/PolinaVorobeva/Homeworks/blob/master/TalentCaseSber_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ЗАГРУЗКА

In [ ]:
!pip install dostoevsky

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.4-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4393296 sha256=b8e53dd289c116b6a723222c8f7417e00bd2fdc1e2a843329110f64191e1a76f
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [ ]:
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 55.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=073552ded33f61e7a57056c907b4e52e123218084fc80dd635dce02468ba2417
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 87.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 134.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 91.3 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=de07c283e7a8fb32ff16c30d1f1d4a8c88fc0967355f15fc0919caf77dbc289f
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [ ]:
!pip install circlify

In [ ]:
!python -m dostoevsky download fasttext-social-network-model

In [ ]:
import pandas as pd
import numpy as np
import pymorphy2
import re
import circlify
import matplotlib.pyplot as plt
%matplotlib inline

import dostoevsky
import nltk
nltk.download("stopwords")

from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

##Для всех карт одна предобученная моделька
tokenizer = RegexTokenizer()
model = FastTextSocialNetworkModel(tokenizer=tokenizer)

##или такая предобученная моделька
from sentence_transformers import SentenceTransformer
# model_stransf = SentenceTransformer('bert-base-nli-mean-tokens')
model_stransf = SentenceTransformer('bert-base-multilingual-cased')


vectorizer = TfidfVectorizer()

from collections import Counter
from wordcloud import WordCloud

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# ФУНКЦИИ

### Предобработка

In [ ]:
def preprocessing_defs_odnoklassniki(df_PRACTISE):
  # Превращаем колонку лайков в числа и заполняем 0 пропуски
  df_PRACTISE['likes'] = df_PRACTISE['likes'].str.lower().str.replace('\n', ' ', regex = False).str.replace(r'[^\d]+', '', regex=True)
  df_PRACTISE['likes'] = pd.to_numeric(df_PRACTISE['likes'])
  df_PRACTISE['likes'] = df_PRACTISE['likes'].fillna(0)
  return df_PRACTISE

def preprocessing_texts(df_PRACTISE):
  # Обрабатываем тексты
  df_PRACTISE['text'] = df_PRACTISE['text'].str.lower().str.replace('\n', ' ', regex = False).str.replace(r'[^\w\s\.\?,!:;-]+', '', regex=True).str.strip()
  # Удаляем получившиеся полностью пустые комментарии, они нам все равно не помогут
  df_PRACTISE = df_PRACTISE[df_PRACTISE['text'].astype(bool) == True]
  # Удаляем пропуски в колонке тексте
  df_PRACTISE.dropna(subset=['text'], inplace=True)

  return df_PRACTISE

### Для статистики

In [ ]:
def remove_stopwords_normalize(TEXT_SERIES):
  clean_text = []
  for string in TEXT_SERIES.str.split(' '):
    for word in string:
      if (word not in russian_stopwords):
        # print(f"RAW FORM IS...{word}")
        word = morph.normal_forms(word)[0]
        # print(f"NORMAL FORM IS...{word}")
        clean_text.append(word)
    text = " ".join(x for x in clean_text)
  return clean_text, text

In [ ]:
def preprocessing_texts_raw(TEXT_SERIES):
  TEXT_SERIES = TEXT_SERIES.str.lower().str.replace('\n', ' ', regex = False).str.replace(r'[^\w\s]+', '', regex=True).str.strip()
  # TEXT_SERIES = TEXT_SERIES.astype(bool) == True
  TEXT_SERIES = TEXT_SERIES.dropna()
  TEXT_SERIES = TEXT_SERIES.apply(lambda x: re.sub(' +', ' ', x))
  raw_text = remove_stopwords_normalize(TEXT_SERIES)

  return raw_text

### Извлечение постов

In [ ]:
def get_posts(df_PRACTISE, column_with_ids, num_of_posts=2, start_post=0):
  posts = df_PRACTISE[column_with_ids].value_counts()
  print(posts)
  ids_for_cognitive_maps = posts.index[start_post:start_post+num_of_posts]

  work = []
  for i in range(num_of_posts):
    post = df_PRACTISE.loc[df_PRACTISE[column_with_ids] == ids_for_cognitive_maps[i]]
    work.append(post)
  return work

### Когнитивное картирование

In [ ]:
def sentiments(text):
  results = model.predict(text, k = 1)
  sentiments = []
  for r in results:
    sent = list(r.keys())[0]
    if (sent == 'speech'): sent = 'neutral'
    sentiments.append(sent)
  return sentiments

In [ ]:
def semantic_similarity(text):
  sentembedding = model_stransf.encode(text.to_list())
  scores = np.zeros((sentembedding.shape[0], sentembedding.shape[0]))

  for i in range(sentembedding.shape[0]):
    scores[i] = cosine_similarity([sentembedding[i]], sentembedding)

  # means = []
  # for sim in scores:
  #   means.append(sim.mean())
  # MIN_SIMILARITY = np.mean(means)
  # print("MEAN SIMILARITY NOW IS ", MIN_SIMILARITY)

  return scores

In [ ]:
def generate_graph(df_PRACTISE, SCORES, SIMILARITY=0.5):
  GRAPH = pd.DataFrame(columns = ['source', 'source label', 'target',
                                  'target label', 'sentiment', 'distance'])
  count = 0
  for i in range (0, len(SCORES)):
    comment_similarities = SCORES[i]
    k = i+1
    for sim in comment_similarities[i+1:]:
      if sim > SIMILARITY:
        count = count + 1
        source = min(df_PRACTISE['time'].iloc[i], df_PRACTISE['time'].iloc[k])
        target = max(df_PRACTISE['time'].iloc[i], df_PRACTISE['time'].iloc[k])
        source_row = df_PRACTISE.query(f'time == {source}')
        target_row = df_PRACTISE.query(f'time == {target}')
        # print(source_row['text'].iloc[0])
        GRAPH = GRAPH.append({'source': source, 'source label': source_row['text'].iloc[0],
                            'target': target, 'target label': target_row['text'].iloc[0],
                          'sentiment':  source_row['sentiment'].iloc[0],
                           'distance': sim}, ignore_index=True)
      k += 1
  print(f"GRAPH HAS... {count} EDGES")
  return GRAPH

In [ ]:
def cognitive_maps_pipline(df_PRACTISE, SIMILARITY=0.5):
  df_PRACTISE = df_PRACTISE.sort_values('likes', ascending=False)

  #эмоциональная окраска
  df_PRACTISE['sentiment'] = sentiments(df_PRACTISE['text'])

  #семантическая близость
  scores = semantic_similarity(df_PRACTISE['text'])

  # непосредственно граф
  GRAPH = generate_graph(df_PRACTISE, scores, SIMILARITY)

  return df_PRACTISE, GRAPH

### Форматы сохранения

In [ ]:
def save_Gephi_format(practise_name, df_PRACTISE, GRAPH):
  VERTECES = {'Id': df_PRACTISE['time'], 'Label': df_PRACTISE['text'], 'Sentiment': df_PRACTISE['sentiment']}
  VERTECES = pd.DataFrame(data = VERTECES)
  VERTECES.to_csv(f'verteces_{practise_name}.csv', index = False)

  EDGES = {'Source': GRAPH['source'],'Target': GRAPH['target'],
         'Weight': GRAPH['distance'],'Type': "direct"}
  EDGES = pd.DataFrame(data = EDGES)
  EDGES.to_csv(f'edges_{practise_name}.csv', index = False)

In [ ]:
#не работает
def save_GC_format(practise_name, df_PRACTISE, GRAPH):
  VERTECES = {'Node Type': df_PRACTISE['time'], 'Name': df_PRACTISE['text']}
  VERTECES = pd.DataFrame(data = VERTECES)

  VERTECES.to_csv(f'verteces_GC_{public_name}.csv', encoding="utf-8")

  EDGES = {'From Type': GRAPH['source'], 'From Name': GRAPH['source label'],
         'Edge Type': GRAPH['sentiment'],
         'To Type': GRAPH['target'], 'To Name': GRAPH['target label'],
         'Weight': GRAPH['distance']}
  EDGES = pd.DataFrame(data = EDGES)
  EDGES.to_csv(f'edges_GC_{public_name}.csv', encoding="utf-8")

#КАРТЫ

## Важная статистика

Количество ребер: 23 [0.965], 17 [0.977], 20 [0.98], 20 [0.96], 17 [0.9798], 18 [0.977],
 17 [0.979], 23 [0.975], 17 [0.94], 17 [0.89]


 Количество ребер  с другой штукой: 18 [0.775], 15 [0.86], 19 [0.86], 16 [0.83], 16 [0.87], 19 [0.86],
 17 [0.92], 18 [0.89], 16 [0.74], 18 [0.75]


## СПОРТ_ОДНОКЛАССНИКИ_2

#### загрузка

In [ ]:
#загрузка
public_name = "спорт_одноклассники_2"
sports0 = pd.read_csv(f"{public_name}.csv", header=0, names=["time","username", "date",
                                                              "text", "likes"])

#### предобработка

In [ ]:
# предобработка
sports0 = preprocessing_defs_odnoklassniki(sports0)
sports0 = preprocessing_texts(sports0)

<ipython-input-9-4f17b0a815ba>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_PRACTISE.dropna(subset=['text'], inplace=True)


#### картирование

In [ ]:
#картирование
sports0, sports0_graph = cognitive_maps_pipline(sports0, 0.775)

MEAN SIMILARITY NOW IS  0.636845280993576
GRAPH HAS... 18 EDGES


<ipython-input-21-b862181dc70c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GRAPH = GRAPH.append({'source': source, 'source label': source_row['text'].iloc[0],
<ipython-input-21-b862181dc70c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GRAPH = GRAPH.append({'source': source, 'source label': source_row['text'].iloc[0],
<ipython-input-21-b862181dc70c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GRAPH = GRAPH.append({'source': source, 'source label': source_row['text'].iloc[0],
<ipython-input-21-b862181dc70c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GRAPH = GRAPH.append({'source': source, 'source label': source_r

#### Формат сохранения

In [ ]:
#создание документов с графами
save_Gephi_format(public_name, sports0, sports0_graph)
# save_GC_format(public_name, practise, graph)

## СПОРТ_ОДНОКЛАССНИКИ

#### загрузка

In [ ]:
#загрузка
public_name = "спорт_одноклассники"
sports1 = pd.read_csv(f"{public_name}.csv", header=0, names=["time","username",
                                                              "text", "likes"])

#### предобработка

In [ ]:
# предобработка

sports1 = preprocessing_defs_odnoklassniki(sports1)
sports1 = preprocessing_texts(sports1)

<ipython-input-7-7fd52da3ef66>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_PRACTISE.dropna(subset=['text'], inplace=True)


#### картирование

In [ ]:
#картирование

sports1, sports1_graph = cognitive_maps_pipline(sports1, 0.86)

MEAN SIMILARITY NOW IS  0.7032187128067017
GRAPH HAS... 15 EDGES


<ipython-input-21-b862181dc70c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GRAPH = GRAPH.append({'source': source, 'source label': source_row['text'].iloc[0],
<ipython-input-21-b862181dc70c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GRAPH = GRAPH.append({'source': source, 'source label': source_row['text'].iloc[0],
<ipython-input-21-b862181dc70c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GRAPH = GRAPH.append({'source': source, 'source label': source_row['text'].iloc[0],
<ipython-input-21-b862181dc70c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GRAPH = GRAPH.append({'source': source, 'source label': source_r

#### формат сохранения

In [ ]:
save_Gephi_format(public_name, sports1, sports1_graph)
# save_GC_format(public_name, practise, graph)

## Юридические консультации

#### загрузка

In [ ]:
#загрузка
name_practise = "Юридические консультации_Артём"
law = pd.read_excel(f'{name_practise}.xlsx')
law = law.rename({'date': 'time'}, axis=1)

post_law0, post_law1 = get_posts(law, 'post_id', 2)

#### предобработка

In [ ]:
# предобработка

post_law0 = preprocessing_texts(post_law0)
post_law1 = preprocessing_texts(post_law1)

<ipython-input-7-7fd52da3ef66>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_PRACTISE['text'] = df_PRACTISE['text'].str.lower().str.replace('\n', ' ', regex = False).str.replace(r'[^\w\s\.\?,!:;-]+', '', regex=True).str.strip()
<ipython-input-7-7fd52da3ef66>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_PRACTISE['text'] = df_PRACTISE['text'].str.lower().str.replace('\n', ' ', regex = False).str.replace(r'[^\w\s\.\?,!:;-]+', '', regex=True).str.strip()


#### картирование

In [ ]:
#картирование

law0, law0_graph = cognitive_maps_pipline(post_law0, 0.86)


MEAN SIMILARITY NOW IS  0.7322284583630921
GRAPH HAS... 19 EDGES


<ipython-input-21-b862181dc70c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GRAPH = GRAPH.append({'source': source, 'source label': source_row['text'].iloc[0],
<ipython-input-21-b862181dc70c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GRAPH = GRAPH.append({'source': source, 'source label': source_row['text'].iloc[0],
<ipython-input-21-b862181dc70c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GRAPH = GRAPH.append({'source': source, 'source label': source_row['text'].iloc[0],
<ipython-input-21-b862181dc70c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GRAPH = GRAPH.append({'source': source, 'source label': source_r

In [ ]:
law1, law1_graph = cognitive_maps_pipline(post_law1, 0.83)

MEAN SIMILARITY NOW IS  0.6269962565657589
GRAPH HAS... 16 EDGES


<ipython-input-21-b862181dc70c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GRAPH = GRAPH.append({'source': source, 'source label': source_row['text'].iloc[0],
<ipython-input-21-b862181dc70c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GRAPH = GRAPH.append({'source': source, 'source label': source_row['text'].iloc[0],
<ipython-input-21-b862181dc70c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GRAPH = GRAPH.append({'source': source, 'source label': source_row['text'].iloc[0],
<ipython-input-21-b862181dc70c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GRAPH = GRAPH.append({'source': source, 'source label': source_r

#### формат сохранения

In [ ]:
save_Gephi_format(name_practise+'0', law0, law0_graph)
save_Gephi_format(name_practise+'1', law1, law1_graph)

## Город проживания

#### загрузка

In [ ]:
# загрузка

name_practise = "Город проживания (СПб и Киров) + юридическая Беларусь"
living = pd.read_excel(f'{name_practise}.xlsx')
living = living.rename({'date': 'time'}, axis=1)

living0, living1 = get_posts(living, 'post_id', 2)

#### предобработка

In [ ]:
# предобработка

living0 = preprocessing_texts(living0)
living1 = preprocessing_texts(living1)

<ipython-input-8-4f17b0a815ba>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_PRACTISE['text'] = df_PRACTISE['text'].str.lower().str.replace('\n', ' ', regex = False).str.replace(r'[^\w\s\.\?,!:;-]+', '', regex=True).str.strip()
<ipython-input-8-4f17b0a815ba>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_PRACTISE.dropna(subset=['text'], inplace=True)
<ipython-input-8-4f17b0a815ba>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

#### картирование

In [ ]:
#картирование
living0, living0_graph = cognitive_maps_pipline(living0, 0.87)

GRAPH HAS... 16 EDGES


<ipython-input-14-b862181dc70c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GRAPH = GRAPH.append({'source': source, 'source label': source_row['text'].iloc[0],
<ipython-input-14-b862181dc70c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GRAPH = GRAPH.append({'source': source, 'source label': source_row['text'].iloc[0],
<ipython-input-14-b862181dc70c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GRAPH = GRAPH.append({'source': source, 'source label': source_row['text'].iloc[0],
<ipython-input-14-b862181dc70c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GRAPH = GRAPH.append({'source': source, 'source label': source_r

In [ ]:
living1, living1_graph = cognitive_maps_pipline(living1, 0.86)

GRAPH HAS... 19 EDGES


<ipython-input-14-b862181dc70c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GRAPH = GRAPH.append({'source': source, 'source label': source_row['text'].iloc[0],
<ipython-input-14-b862181dc70c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GRAPH = GRAPH.append({'source': source, 'source label': source_row['text'].iloc[0],
<ipython-input-14-b862181dc70c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GRAPH = GRAPH.append({'source': source, 'source label': source_row['text'].iloc[0],
<ipython-input-14-b862181dc70c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GRAPH = GRAPH.append({'source': source, 'source label': source_r

#### формат сохранения

In [ ]:
save_Gephi_format(name_practise+'0', living0, living0_graph)
save_Gephi_format(name_practise+'1', living1, living1_graph)

## Услуги

#### загрузка

In [ ]:
# загрузка
name_practise = "Услуги class domklining_belkacar_yandexgo"
services = pd.read_excel(f'{name_practise}.xlsx', index_col=None, header=None,
                         names = ['vertex', 'time', 'text', 'class', 'likes', 'post_id'])

_, services1, services2 = get_posts(services, 'post_id', 3)

services1['vertex'] = services1['vertex'].astype(int)
services2['vertex'] = services2['vertex'].astype(int)

<ipython-input-24-f21dd97d0ad0>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  services1['vertex'] = services1['vertex'].astype(int)
<ipython-input-24-f21dd97d0ad0>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  services2['vertex'] = services2['vertex'].astype(int)


#### предобработка

In [ ]:
# предобработка

services1 = preprocessing_texts(services1)
services2 = preprocessing_texts(services2)

<ipython-input-7-7fd52da3ef66>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_PRACTISE['text'] = df_PRACTISE['text'].str.lower().str.replace('\n', ' ', regex = False).str.replace(r'[^\w\s\.\?,!:;-]+', '', regex=True).str.strip()
<ipython-input-7-7fd52da3ef66>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_PRACTISE.dropna(subset=['text'], inplace=True)
<ipython-input-7-7fd52da3ef66>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

#### картирование

In [ ]:
# картирование

services1, services1_graph = cognitive_maps_pipline(services1, 0.92)

MEAN SIMILARITY NOW IS  0.6649953448648367
GRAPH HAS... 17 EDGES


<ipython-input-21-b862181dc70c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GRAPH = GRAPH.append({'source': source, 'source label': source_row['text'].iloc[0],
<ipython-input-21-b862181dc70c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GRAPH = GRAPH.append({'source': source, 'source label': source_row['text'].iloc[0],
<ipython-input-21-b862181dc70c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GRAPH = GRAPH.append({'source': source, 'source label': source_row['text'].iloc[0],
<ipython-input-21-b862181dc70c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GRAPH = GRAPH.append({'source': source, 'source label': source_r

In [ ]:
services2, services2_graph = cognitive_maps_pipline(services2, 0.89)

MEAN SIMILARITY NOW IS  0.6812896870374681
GRAPH HAS... 18 EDGES


<ipython-input-21-b862181dc70c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GRAPH = GRAPH.append({'source': source, 'source label': source_row['text'].iloc[0],
<ipython-input-21-b862181dc70c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GRAPH = GRAPH.append({'source': source, 'source label': source_row['text'].iloc[0],
<ipython-input-21-b862181dc70c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GRAPH = GRAPH.append({'source': source, 'source label': source_row['text'].iloc[0],
<ipython-input-21-b862181dc70c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GRAPH = GRAPH.append({'source': source, 'source label': source_r

#### формат сохранения

In [ ]:
save_Gephi_format(name_practise+'1', services1, services1_graph)
save_Gephi_format(name_practise+'2', services2, services2_graph)

## Здоровое питание

#### загрузка

In [ ]:
# загрузка
public_name = "ЗдоровоеПитание"
healthy = pd.read_csv(f"{public_name}.csv", header=0, sep=';')
healthy = healthy.rename({'date': 'time'}, axis=1)

healthy0, healthy1 = get_posts(healthy, 'post_id')

1074959    19
496526     19
1102109    17
1080785    16
1074619    13
           ..
490870      1
1093290     1
497328      1
1093590     1
"6"         1
Name: post_id, Length: 528, dtype: int64


#### предобработка

In [ ]:
# предобработка

healthy0 = preprocessing_texts(healthy0)
healthy1 = preprocessing_texts(healthy1)

<ipython-input-29-4f17b0a815ba>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_PRACTISE['text'] = df_PRACTISE['text'].str.lower().str.replace('\n', ' ', regex = False).str.replace(r'[^\w\s\.\?,!:;-]+', '', regex=True).str.strip()
<ipython-input-29-4f17b0a815ba>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_PRACTISE['text'] = df_PRACTISE['text'].str.lower().str.replace('\n', ' ', regex = False).str.replace(r'[^\w\s\.\?,!:;-]+', '', regex=True).str.strip()


In [ ]:
len(healthy0)

19

#### картирование

In [ ]:
# картирование

healthy0, healthy0_graph = cognitive_maps_pipline(healthy0, 0.74)


GRAPH HAS... 16 EDGES


<ipython-input-14-b862181dc70c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GRAPH = GRAPH.append({'source': source, 'source label': source_row['text'].iloc[0],
<ipython-input-14-b862181dc70c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GRAPH = GRAPH.append({'source': source, 'source label': source_row['text'].iloc[0],
<ipython-input-14-b862181dc70c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GRAPH = GRAPH.append({'source': source, 'source label': source_row['text'].iloc[0],
<ipython-input-14-b862181dc70c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GRAPH = GRAPH.append({'source': source, 'source label': source_r

In [ ]:
healthy1, healty1_graph = cognitive_maps_pipline(healthy1, 0.75)

GRAPH HAS... 18 EDGES


<ipython-input-14-b862181dc70c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GRAPH = GRAPH.append({'source': source, 'source label': source_row['text'].iloc[0],
<ipython-input-14-b862181dc70c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GRAPH = GRAPH.append({'source': source, 'source label': source_row['text'].iloc[0],
<ipython-input-14-b862181dc70c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GRAPH = GRAPH.append({'source': source, 'source label': source_row['text'].iloc[0],
<ipython-input-14-b862181dc70c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GRAPH = GRAPH.append({'source': source, 'source label': source_r

#### формат сохранения

In [ ]:
save_Gephi_format(public_name+'0', healthy0, healthy0_graph)
save_Gephi_format(public_name+'1', healthy1, healty1_graph)